# 05. 벡터의 유사도(Vector Similarity)
* 문서의 유사도 구하기
* 주로 문서들 간에 동일한 단어 또는 비슷한 단어가 얼마나 ㅈ공통적으로 많이 사용되었는지
* 기계가 계산하는 문서의 유사도 성능: 
    * 각 문서의 단어들을 어떤 방법으로 수치화하여 표현했는지(DTM, Word2Vec)
    * 문서 간의 단어들의 차이를 어떤 방법(유클리드 거리, 코사인 유사도)등으로 계산했는지

## 1) 코사인 유사도(Cosine similarity)
* BoW, BoW 기반 단어 표현 방법인 DTM, TD-IDF 또는 뒤에서 배우게 될 워드투벡터 등 단어를 수치화할 수 있는 방법
* 이러한 표현 방법에 대해서 코사인 유사도를 이용하여 문서의 유사도 구하기

### 1. 코사인 유사도(Cosine Similarity)
* 두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도
    * 방향 완전 동일: 1, 90도: 0, 180도: -1
    * -1~1, 값이 1에 가까울 수록 유사도가 높다
    * 식: similarity = cos(Θ) = (A ⋅ B) / (||A|| ||B||)
        

* 문서 단어 행렬이나 TF-IDF 행렬을 통해서 문서의 유사도를 구하는 경우에는 문서 단어 행렬이나 TF-IDF 행렬이 각각의 특징벡터 A, B가 됨
* 문서 단어 행렬에 대해 코사인 유사도를 구해 봄(numpy 이용)

In [2]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

# 코사인 유사도 구하는 함수
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

# 문서 1, 2, 3에 대하여 각각 BoW 만듦
doc1 = np.array([0, 1, 1, 1])
doc2 = np.array([1, 0, 1, 1])
doc3 = np.array([2, 0, 2, 2])

# 각 문서에 대한 코사인 유사도
print(cos_sim(doc1, doc2))
print(cos_sim(doc1, doc3))
print(cos_sim(doc2, doc3)) # 1은 두 벡터의 방향이 완전히 동일한 경우

0.6666666666666667
0.6666666666666667
1.0000000000000002


* 문서3은 문서 2에서 모든 단어의 빈도수가 1씩 증가했을 뿐
* 즉, 한 문서 내의 모든 단어의 빈도수가 동일하게 증가하는 경우, 기존의 문서와 코사인 유사도의 값이 1
* ==> 코사인 유사도는 문서의 길이가 다른 사오하엥서 비교적 공정한 비교를 할 수 있도록 함
    * b/c 유사도를 구할 때, 벡터의 크기가 아니라 벡터의 방향(패턴)에 초점을 두기 때문
    * 벡터의 유사도를 구하는 또 다른 방법인 내적과의 차이

### 2. 유사도를 이용한 추천 시스템 구현하기
* 캐글 영화 데이터셋으로 영화 추천 시스템
* TF-IDF와 코사인 유사도 만으로 영화의 줄거리에 기반해서 영화를 추천하는 추천 시스템 가능

In [4]:
import pandas as pf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd

data = pd.read_csv('C:\\Users\\user\\Documents\\movies_metadata.csv', low_memory=False)
data.head(5)

# 코사인 유사도에 사용할 데이터: 영화 제목에 해당하는 title열과 줄거리에 해당하는 overview열

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
data = data.head(20000)

In [6]:
# tf-idf를 할 때 데이터에 null값이 있으면 안 되므로 확인해봄
data['overview'].isnull().sum()

135

In [7]:
#overview에서 null값을 가진 경우에는 빈값으로 대테하여 null값을 제거
data['overview'] = data['overview'].fillna('')

In [8]:
# tf-idf 수행
tfidf = TfidfVectorizer(stop_words='english')
# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

(20000, 47487)


In [9]:
# 코사인 유사도
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 영화의 타이틀과 인덱스를 가진 테이블 만듦
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [10]:
# 영화의 타이틀을 입력하면 인덱스를 리턴하기 위함
idx = indices['Father of the Bride Part II']
print(idx)

4


In [12]:
# 선택한 영화에 대해서 코사인 유사도를 이용하여, 가장 overview가 유사한 10개의 영화를 찾아내는 함수
def get_recommendations(title, cosine_sim=cosine_sim):
    #선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옴. 이제 선택한 영화를 가지고 연산 가능
    idx = indices[title]
    
    # 모든 영화에 대해서 해당 영화와의 유사도를 구함
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # 유사도에 따라 영화 정렬
    sim_scores = sorted(sim_scores, key= lambda x: x[1], reverse=True)
    
    # 가장 유사한 10개의 영화를 받아옴
    sim_scores = sim_scores[1:11]
    
    # 가장 유사한 10개의 영화의 인덱스를 받아옴
    movie_indices = [i[0] for i in sim_scores]
    
    # 가장 유사한 10개의 영화의 제목 리턴
    return data['title'].iloc[movie_indices]

In [13]:
get_recommendations('The Dark Knight Rises')

# 가장 유사한 영화가 출력됨. 영화 다크 나이트 첫번째, 그 외에도 전부 배트맨 영화를 찾아낸 것을 확인할 수 있음

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

## 2) 여러가지 유사도 기법
### 1. 유클리드 거리(Euclidean distance)
* 문서 유사도 구할 때 자카드 유사도나 코사인 유사도만큼 유용하지는 않지만 의미 있음
    * root((q1−p1)2+(q2−p2)2+ ... +(qn−pn)2)
* ex) 바나나 사과 저는 좋아요 / 문서1, 문서2, 문서3 있음.
    * 단어 개수 4개이므로 4차원 공간에 문서1, 문서2, 문서3을 배치하는 것과 같음
    * 문서Q에 대해 문서1, 2, 3 중 가장 유사한 문서 찾아내고자 함
        * 유클리드 거리를 이용한다면 문서Q도 다른 문서들처럼 4차원 공간에 배치시켰다는 관점에서 4차원 공간에서의 각각의 문서들과의 유클리드 거리 구하면 됨

In [17]:
import numpy as np
def dist(x, y):
    return np.sqrt(np.sum((x-y)**2))

doc1 = np.array((2, 3, 0, 1))
doc2 = np.array((1, 2, 3, 1))
doc3 = np.array((2, 1, 2, 2))
docq = np.array((1, 1, 0, 1))

In [18]:
print(dist(doc1, docq))
print(dist(doc2, docq))
print(dist(doc3, docq))

2.23606797749979
3.1622776601683795
2.449489742783178


* **유클리드 거리 값이 작을수록 문서 간의 거리 가까움--> 문서1이 문서q와 가장 유사**

### 2. 자카드 유사도(Jaccard similarity)
* A와 B 두 개의 집합의 교집합(두 개의 집합에서 공통으로 가지는 원소들의 집합)
* 자카드 유사도: 합집합에서 교집합의 비율을 구한다면 두 집합 A와 B의 유사도를 구할 수 있음
* 0-1 사이의 값. 두 집합이 동일하다면 1, 없다면 0
* 식: J(A, B) = |A∩B|/|A∪B| = |A∩B|/(|A|+|B|-|A∩B|)

In [21]:
# 두 개의 문서. 두 문서 모두에게 등장한 단어는 apple과 banana 두 개
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화를 수행
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

# 토큰화 결과 출력
print(tokenized_doc1)
print(tokenized_doc2)

['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [22]:
# 문서1과 문서2의 합집합
union = set(tokenized_doc1).union(set(tokenized_doc2))
print(union)

{'banana', 'passport', 'love', 'you', 'everyone', 'apple', 'holder', 'coupon', 'likey', 'like', 'card', 'watch'}


In [23]:
# 교집합
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print(intersection)

{'banana', 'apple'}


In [24]:
print(len(intersection)/len(union))

0.16666666666666666
